In [1]:
import findspark
findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .master("spark://namenode:7077")\

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-880150c4-2232-4122-ab35-9d86e93e1b84;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 240ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

In [2]:
spark.stop()

In [2]:
df = spark.read.format("delta").load("/temp/filtered_reviews")

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 2]
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [3]:
df.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|WI2cpA5VSEgGE5Pl9...|pym7c6ZFEtmoH16xN...|We saw the review...|2019-10-04 11:50:29|Restaurants, Bars...|  4.0|
|JMm_2beO-LpbNc4r6...|Ei5HBqe012ImhqEr2...|This is a delight...|2019-09-08 21:52:26|Italian, Restaura...|  5.0|
|bWk_7CwRfQpdkZMMM...|UmjITdXHhEF46ho6I...|This place is FUC...|2019-04-14 04:48:15|Adult Entertainme...|  1.0|
|bBqyHGJpbjp68gmmP...|hy5GpGXAna-5qrb3z...|Literally disgust...|2019-05-20 22:54:11|Mexican, Restaura...|  1.0|
|SH_DWH_hzRBTc2TRK...|vN6v8m4DO45Z4pp8y...|Surrey's is great...|2013-01-07 02:00:05|Vegetarian, Resta...|  4.0|
+--------------------+--------------------+--------------------+-------------------+--------------------

### Removing Links
First lets see if links are even used in Yelp reviews.

In [4]:
df.where(F.col('text').contains('https')).take(5)

[Row(review_id='FAjTzZf7ifNbJxp2yhTgLA', business_id='OM5beWRkxZSu1TShUC3EDw', text='It was a tasty treat indeed. I ate it as recommended by Trevor with the bib and everything. The saucy was super butter 9000 and I would totally eat that this joint again for sure. Even now my fingers and bread smell of the glorious butter deliciousness. Here is my live yelp https:\\/\\/www.youtube.com\\/watch?v=9dmSN5IfU9Y', date='2017-10-03 03:40:46', categories='Food, Specialty Food, Seafood, Cajun\\/Creole, Restaurants', stars='5.0'),
 Row(review_id='DGnzpZlqRdzNQ3rgBRE9Ng', business_id='AueGCAS9RtM2RU19euf49w', text="We have our weekly family (usually Saturday night's) outing at Giuseppe's, here in Lambertville, New Jersey, and Joanne and the staff always provide a pleasant place for us to gather - with their BYOB policy, we enjoy a festive time, excellent background music (usually a nice mix of old classics), and of-course great pizza (from white pizza, vegetarian pizza, and regular red pizza) wit

In [6]:
df2 = df.withColumn('text', F.regexp_replace('text', r'http[s]?:\S+|www\.[^\s]*', '')) #remove links

Lets check if the links are gone

In [7]:
df2.where(F.col('text').contains('https')).take(5)

[Row(review_id='dRjRdepVvaSZM7bJEGifcw', business_id='peeXG1Ra-DClPQv14jvKvg', text="Just had my first big Chinese food craving since moving to Philly. Lucky for me I had you yelpers to help me pick the perfect spot to satisfy that need.I ordered online, which was easy and comfortable. Only weird thing was that when I went to pay, there wasn't a secure (https) link where you enter your credit card info. I'm paranoid, so cash it was.I was impressed at how fast the food came. It was less than 30 min after I submitted my order that a nice delivery person was knocking on my door in the art museum area. Given that it was a rainy Saturday night, I was pleased with the timing.The food arrived hot -- not lame-o lukewarm like some delivery places. The portions were large and the food was tasty. We always order way (I mean WAY) more than necessary so we tried and enjoyed all of the following:pot stickers (chicken, steamed, and very tender), beef with broccoli (crunchy broccoli - yay!!) (asked fo

This text contains not a link but a mention of the https protocol.

We can also check how one of the examples above looks now

In [8]:
df2.where(F.col('review_id').contains('FAjTzZf7ifNbJxp2yhTgLA')).take(1)

[Row(review_id='FAjTzZf7ifNbJxp2yhTgLA', business_id='OM5beWRkxZSu1TShUC3EDw', text='It was a tasty treat indeed. I ate it as recommended by Trevor with the bib and everything. The saucy was super butter 9000 and I would totally eat that this joint again for sure. Even now my fingers and bread smell of the glorious butter deliciousness. Here is my live yelp ', date='2017-10-03 03:40:46', categories='Food, Specialty Food, Seafood, Cajun\\/Creole, Restaurants', stars='5.0')]

Before: '... Here is my live yelp https:\\/\\/www.youtube.com\\/watch?v=9dmSN5IfU9Y'

After: '... Here is my live yelp '

In [12]:
df2.summary().show()

+-------+--------------------+--------------------+----------------------+----------+--------------------+------------------+
|summary|           review_id|         business_id|                  text|      date|          categories|             stars|
+-------+--------------------+--------------------+----------------------+----------+--------------------+------------------+
|  count|             4724468|             4724468|               4724468|   4724376|             4724468|           4724468|
|   mean|                null|                null|                  null|      null|                null|3.7937981588614846|
| stddev|                null|                null|                  null|      null|                null|1.3912277728543867|
|    min|---4VcQZzy_vIIifU...|---kPU91CF4Lq2-Wl...|                      |          |Acai Bowls, Ameri...|               1.0|
|    25%|                null|                null|                  null|      null|                null|            

In [13]:
fixedStarsDf = df2.withColumn('stars', F.col('stars').cast('double'))

In [15]:
fixedStarsDf.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|WI2cpA5VSEgGE5Pl9...|pym7c6ZFEtmoH16xN...|We saw the review...|2019-10-04 11:50:29|Restaurants, Bars...|  4.0|
|JMm_2beO-LpbNc4r6...|Ei5HBqe012ImhqEr2...|This is a delight...|2019-09-08 21:52:26|Italian, Restaura...|  5.0|
|bWk_7CwRfQpdkZMMM...|UmjITdXHhEF46ho6I...|This place is FUC...|2019-04-14 04:48:15|Adult Entertainme...|  1.0|
|bBqyHGJpbjp68gmmP...|hy5GpGXAna-5qrb3z...|Literally disgust...|2019-05-20 22:54:11|Mexican, Restaura...|  1.0|
|SH_DWH_hzRBTc2TRK...|vN6v8m4DO45Z4pp8y...|Surrey's is great...|2013-01-07 02:00:05|Vegetarian, Resta...|  4.0|
+--------------------+--------------------+--------------------+-------------------+--------------------

In [18]:
trainDF, testDF = fixedStarsDf.randomSplit([.8, .2], seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 3780240 rows in the training set, and 944228 in the test set


In [20]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[tokenizer, hashtf, idf])

In [19]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["text"], outputCol="features")

vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF.select("text", "features", "stars").show(10)

IllegalArgumentException: Data type string of column text is not supported.

###  Lets check that classes are similarly distributed

There are moslty positive reviews with 5 and 4 stars. 1, 2 and 3 star reviewes make up only 1/2 of the dataset

In [11]:
fractions = df2.select("stars").distinct().withColumn("fraction", F.lit(0.8)).rdd.collectAsMap()


In [12]:
fractions

{'1.0': 0.8, '5.0': 0.8, '4.0': 0.8, '2.0': 0.8, '3.0': 0.8}

In [13]:
train = df2.sampleBy('stars', fractions, seed = 1)

In [14]:
test = df2.subtract(train)

Sample the same amount of data from the dataset

In [15]:
train.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|5KBe-TUWtawFwuew4...|uV7Zc9zIgDG4t2O_t...|We order the same...|2017-06-28 22:01:04|Mexican, Restaura...|  5.0|
|rrNpGRFH2xQ2Pajo7...|H2Kulc2EWVO0kFmAD...|Ordered 'The Spic...|2016-05-20 03:16:25|Sandwiches, Resta...|  5.0|
|3S1_0XZIJIpmOlDBX...|ejDrtuWQRGY0a3zjd...|"This was my 5th ...|2013-05-30 18:14:20|Nightlife, Americ...|  2.0|
|hNEUx4fOE-Aq_J9OQ...|eQHk2EMRtkYZR6DH-...|"Yelp's phrase fo...|2012-01-06 17:14:57|American (New), B...|  2.0|
|LwQJHKnbtje3LGCCc...|NJaR_2hke7NNAC6l1...|Super friendly st...|2016-05-08 21:50:20|Restaurants, Mexican|  3.0|
+--------------------+--------------------+--------------------+-------------------+--------------------

If we look at the distribution of star values in the training dataset, we can see that we have way more 5 star reviews than for example 2 star reviews. If we would like to get an even distribution we can compute a new facor as follows.

In [35]:
distribution = train.groupBy("stars").count().orderBy('stars')

In [36]:
distribution.show()

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0| 453317|
|  2.0| 323645|
|  3.0| 434598|
|  4.0| 903966|
|  5.0|1663173|
+-----+-------+



In [64]:
min_value = distribution.select(F.min(F.col("count"))).collect()[0][0]

In [65]:
min_value

323645

In [66]:
distribution = distribution.withColumn("count_ratio", min_value / F.col("count"))

In [70]:
even_fractions = distribution.select('stars', 'count_ratio').rdd.collectAsMap()

In [71]:
even_train = train.sampleBy('stars', even_fractions, seed = 1)

In [75]:
even_train.groupBy("stars").count().orderBy('stars').show()

+-----+------+
|stars| count|
+-----+------+
|  1.0|323507|
|  2.0|323645|
|  3.0|323635|
|  4.0|324104|
|  5.0|324124|
+-----+------+



Now all the possible star values have around the same probability.

We can do the same for the test values

In [77]:
distribution = test.groupBy("stars").count().orderBy('stars')
min_value = distribution.select(F.min(F.col("count"))).collect()[0][0]
distribution = distribution.withColumn("count_ratio", min_value / F.col("count"))
even_fractions = distribution.select('stars', 'count_ratio').rdd.collectAsMap()
even_test = test.sampleBy('stars', even_fractions, seed = 1)
even_train.groupBy("stars").count().orderBy('stars').show()


+-----+------+
|stars| count|
+-----+------+
|  1.0|323507|
|  2.0|323645|
|  3.0|323635|
|  4.0|324104|
|  5.0|324124|
+-----+------+



In [113]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [116]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
swr = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashtf = HashingTF(numFeatures=2**16, inputCol="filtered_words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "stars", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, swr, hashtf, idf, label_stringIdx])

In [117]:
pipelineFit = pipeline.fit(even_train)
train_df = pipelineFit.transform(even_train)
val_df = pipelineFit.transform(even_test)
train_df.show(5)

IllegalArgumentException: wofiltered_wordsrds does not exist. Available: review_id, business_id, text, date, categories, stars, words, filtered_words

In [101]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100, class_weight = 'balanced')
lrModel = lr.fit(train_df)

2023-03-31 11:09:26,734 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1099.3 KiB
2023-03-31 11:10:20,040 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1100.5 KiB
2023-03-31 11:10:20,333 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1100.9 KiB
2023-03-31 11:11:18,668 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1102.0 KiB
2023-03-31 11:11:19,138 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1100.9 KiB
2023-03-31 11:11:21,816 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1102.0 KiB
2023-03-31 11:11:22,140 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1100.9 KiB
2023-03-31 11:11:24,296 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1102.0 KiB
2023-03-31 11:11:24,540 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1100.9 KiB
2023-03-31 11:11:26,498 WARN scheduler.DAGScheduler: Br

In [102]:
predictions = lrModel.transform(val_df)

In [103]:
predictions.show(5)

2023-03-31 11:42:57,120 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+--------------------+--------------------+--------------------+-------------------+--------------------+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|           review_id|         business_id|                text|               date|          categories|stars|               words|                  tf|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|1l2b8pEVjmbkfHA48...|u5_y_iOUMjbvqSwWX...|"I went for Resta...|2008-02-23 06:52:12|Italian, Restaurants|  2.0|["i, went, for, r...|(65536,[239,308,3...|(65536,[239,308,3...|  2.0|[-0.0153883566597...|[0.09452295222193...|       4.0|
|3rbxCxwleVSQFDhHd...|CRnqAB-ViEfQ0CQGO...|The bread is very...|

In [104]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator()
evaluator.setPredictionCol("prediction")
evaluator.setLabelCol("label")
evaluator.evaluate(predictions)

2023-03-31 11:45:20,492 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


0.5837941674528926

In [92]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [93]:
predictionAndLabels = predictions.select('prediction','label')

In [95]:
predictionAndLabels.show(5)

2023-03-31 10:58:20,510 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  2.0|
|       3.0|  2.0|
|       3.0|  3.0|
|       1.0|  1.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [97]:
sc.parallelize(predictionAndLabels)

NameError: name 'sc' is not defined

In [96]:
metrics = MulticlassMetrics(predictionAndLabels)

AttributeError: 'DataFrame' object has no attribute 'ctx'

In [86]:
predictions.write.format("delta").mode("overwrite").save("/temp/predicted_reviews")

2023-03-31 10:46:16,703 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [37]:
val_df.show(5)

2023-03-25 23:23:18,685 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1119.3 KiB


+--------------------+--------------------+--------------------+-------------------+--------------------+-----+--------------------+--------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|               words|                  tf|            features|label|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+--------------------+--------------------+--------------------+-----+
|2BC1-gBhvs2Wota17...|zZ01WQlcpI1_n806W...|"Almost too large...|2018-10-26 15:40:33|Restaurants, Amer...|  4.0|["almost, too, la...|(65536,[1198,1257...|(65536,[1198,1257...|  1.0|
|2J2sfhgashGlI-VIE...|hS8z5yNV0QJQcKo1a...|I wrote a review ...|2019-02-27 14:32:03|Pubs, Pizza, Nigh...|  5.0|[i, wrote, a, rev...|(65536,[680,1198,...|(65536,[680,1198,...|  0.0|
|2UQmzimc0yNAHEC4m...|6ajnOk0GcY9xbb5Oc...|I loved my meal a...|2014-05-15 23:10:28|Mediterrane

In [38]:
prediction = lrModel.predict(val_df.head().features)

2023-03-25 23:26:52,791 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1121.9 KiB


In [40]:
prediction

0.0

In [42]:
val_df.head().text

2023-03-25 23:51:31,640 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1121.9 KiB


'"Wow, I am surprised that there are no reviews of this restaurant on Yelp...in fact, it looks like I am the first to write one??!!!! Oh!! Now that\'s pressure.Anyway, this will be an easy review so no pressure here. Antonella\'s is probably one of the better Italian restaurants that I have ever tried.  Each soup, salad, sandwich and platter at this little Italian eatery is magnificent. The service is personal and friendly, and nowhere else can you get better food in a hurry for a great price. The portions are big, the selection is great, and the smell is intoxicating. Watch out for the lunch time rush...call ahead instead. Haven\'t had a bad dish there yet...and they cater for any of your special occasions.My recommendations: Lentil Soup, Eggplant Parmesan, the "Ogelsby" hoagie .All hoagies and sandwiches are terrific...and I\'ve tried almost all of them. So overall here are my final ratings...Food: 5 starsService: 5 starsAmbiance: 3 starsOverall: 5 starsGotta go now, my mouth is wate